In [ ]:
# Example calibration data: 10 samples of 1x3x224x224
calibration_data = [np.random.rand(1, 3, 224, 224).astype(np.float32) for _ in range(10)]
data_reader = MyDataReader(calibration_data)


In [1]:
import numpy as np
import onnx
from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantFormat, QuantType
import librosa as lb
import os
from scripts.processing import predict, load_audio,mel_to_model, wave_to_mel
from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantFormat, QuantType


In [ ]:
#Static

In [9]:
original_model_path = "models/CRNN_pruned_001.onnx"
quantized_model_path = 'models/CRNN_quantized_mixed.onnx'

In [10]:
main_dir="c:/Users/Admin/Documents/RealAudio/audios_ZOOM0"
files=os.listdir(main_dir)
print(len(files))

549


In [20]:
inputs=[]
for f in files:
    Audio_dir =main_dir+"/"+f
    dir_audio=Audio_dir
    sr=16000
    audio=load_audio(dir_audio,sr)

    mel=wave_to_mel(audio)
    mel=mel_to_model(mel,[0,0], yolo=False).astype(np.float32)
    print(mel.shape)

    if mel.shape == (1,128, 626):
        inputs.append(mel)

(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 128, 626)
(1, 12

In [12]:
np.array(inputs).shape

(0,)

In [21]:
class MyDataReader(CalibrationDataReader):
    def __init__(self, calibration_data):
        self.data = calibration_data
        self.iterator = iter(calibration_data)

    def get_next(self):
        try:
            return {"input": next(self.iterator)}
        except StopIteration:
            return None
data_reader = MyDataReader(inputs)


In [22]:
quantize_static(
    model_input=original_model_path,
    model_output=quantized_model_path,
    calibration_data_reader=data_reader,
    quant_format=QuantFormat.QOperator,
    activation_type=QuantType.QUInt8,
    weight_type=QuantType.QUInt8,
    
)

  elem_type: 7
  shape {
  }
}
.
  elem_type: 7
  shape {
    dim {
      dim_value: 1
    }
  }
}
.
  elem_type: 7
  shape {
  }
}
.
  elem_type: 7
  shape {
    dim {
      dim_value: 1
    }
  }
}
.


In [8]:
quantize_static(
    model_input=original_model_path,
    model_output=quantized_model_path,
    calibration_data_reader=data_reader,
    quant_format=QuantFormat.QOperator,
    activation_type=QuantType.QUInt8,
    weight_type=QuantType.QUInt8,
    nodes_to_exclude=[
  "/model.24/Concat_3",
  "/model.24/Split",
  "/model.24/dfl/Reshape",
  "/model.24/dfl/Transpose",
  "/model.24/dfl/Softmax",
  "/model.24/dfl/conv/Conv",
  "/model.24/dfl/Reshape_1",
  "/model.24/Shape",
  "/model.24/Gather",
  "/model.24/Add",
  "/model.24/Div",
  "/model.24/Mul",
  "/model.24/Mul_1",
  "/model.24/Slice",
  "/model.24/Slice_1",
  "/model.24/Sub",
  "/model.24/Add_1",
  "/model.24/Add_2",
  "/model.24/Sub_1",
  "/model.24/Div_1",
  "/model.24/Concat_4",
  "/model.24/Mul_2",
  "/model.24/Concat_5",
  "/model.24/Sigmoid"
    ]
    ,
)

In [254]:
import onnx
import numpy as np
from onnx import numpy_helper

# Cargar el modelo ONNX
original_model_path = "models/YOLOv5n_original.onnx"
quantized_model_path = 'models/YOLOv5n_pruned_001.onnx'


model = onnx.load(original_model_path)


In [255]:

def prune_weights(weights, threshold=0.001):
    mask = np.abs(weights) > threshold
    pruned_weights = weights * mask
    return pruned_weights

# Aplicar poda a los pesos
for node in model.graph.node:
    for input_name in node.input:
        initializer = next((x for x in model.graph.initializer if x.name == input_name), None)
        if initializer is not None:
            weights = numpy_helper.to_array(initializer)
            pruned_weights = prune_weights(weights)
            new_initializer = numpy_helper.from_array(pruned_weights, input_name)
            model.graph.initializer.remove(initializer)
            model.graph.initializer.append(new_initializer)

In [256]:
onnx.save(model, quantized_model_path)